In [ ]:
# Following this Course:
# https://huggingface.co/learn/nlp-course/en/chapter7/2

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e772554eca898dd7321eb09892873dfea5dd5488dc0ef497679da75f42560d06
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!pip install accelerate -U

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import os

path = "/content/drive/MyDrive/NLP_Project_New"
sys.path.append(os.path.abspath(path))

In [ ]:
import nlp_project_functions as functions

from transformers import AutoTokenizer
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import pipeline

import torch
from pathlib import Path
import re
import numpy as np
import evaluate
import pandas as pd

from sklearn.model_selection import train_test_split

In [ ]:
def encode_tags(tags, encodings):
    labels = [[label2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []

    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())
    return encoded_labels

In [ ]:
# create dataset

class SermonDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# model evaluation
metric = evaluate.load("seqeval")

In [ ]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    results = {
        "per_precision": all_metrics["PER"]["precision"],
        "per_recall": all_metrics["PER"]["recall"],
        "per_f1": all_metrics["PER"]["f1"],
        "loc_precision": all_metrics["LOC"]["precision"],
        "loc_recall":all_metrics["LOC"]["recall"],
        "loc_f1": all_metrics["LOC"]["f1"],
        "overall_precision": all_metrics["overall_precision"],
        "overall_recall": all_metrics["overall_recall"],
        "overall_f1": all_metrics["overall_f1"],
        "overall_accuracy": all_metrics["overall_accuracy"],
    }

    return results

In [ ]:
label_names_dict = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-LOC': 3, 'I-LOC': 4}
label_names_list = ["O", "B-PER", "I-PER", "B-LOC", "I-LOC"]

In [ ]:
unique_labels = label_names_list
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

In [ ]:
for i in [2, 4, 8, 16, 32, 64]:
  model_checkpoint = "dbmdz/bert-base-german-cased"

  # tokenize
  tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

  # define the model
  model = AutoModelForTokenClassification.from_pretrained(
      model_checkpoint,
      id2label = id2label,
      label2id = label2id,
  )

  # training and validation data
  texts, tags = functions.read_conll_data(f'{path}/data/train_test_val/train_smaller/{i}_part_train.tsv')
  # split data
  if i <= 8:
    train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)
  else:
    train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.3)

  batch_steps = int((len(train_texts) / 16) * 3)
  eval_steps = int(batch_steps / 4)
  save_steps = eval_steps * 2

  train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=False)
  val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=False)

  train_labels = encode_tags(train_tags, train_encodings)
  val_labels = encode_tags(val_tags, val_encodings)

  train_encodings.pop("offset_mapping") # we don't want to pass this to the model
  val_encodings.pop("offset_mapping")

  train_dataset = SermonDataset(train_encodings, train_labels)
  val_dataset = SermonDataset(val_encodings, val_labels)

  args = TrainingArguments(
      f"{path}/models/bert_finetuned_{i}_part",
      evaluation_strategy="steps",
      save_steps=save_steps,
      eval_steps=eval_steps,
      load_best_model_at_end=True,
      save_total_limit=5,
      learning_rate=2e-5,
      num_train_epochs=3,
      weight_decay=0.01,
      per_device_train_batch_size=16,  # batch size per device during training
      per_device_eval_batch_size=64,   # batch size for evaluation
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
      logging_strategy="steps",
      logging_dir=f'{path}/logs/bert_finetuned_{i}_part',
      logging_steps=200,
  )

  trainer = Trainer(
      model=model,
      args=args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      compute_metrics=compute_metrics
  )

  print(f"Training on slice {i}. Training set: {len(train_texts)}, Validation set: {len(val_texts)}. Eval. steps: {eval_steps}, Save steps: {save_steps}")

  trainer.train()

  trainer.save_model(f"{path}/models/bert_finetuned_{i}_part")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

...

Training on slice 32. Training set: 512, Validation set: 220. Eval. steps: 24, Save steps: 48


Step,Training Loss,Validation Loss,Per Precision,Per Recall,Per F1,Loc Precision,Loc Recall,Loc F1,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
24,No log,1.307100,0.012903,0.078947,0.022181,0.001487,0.125000,0.002939,0.004420,0.086957,0.008412,0.473844
48,No log,0.591087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.976038
72,No log,0.132224,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.981100
96,No log,0.100168,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.981100


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following argum

Training on slice 64. Training set: 256, Validation set: 110. Eval. steps: 12, Save steps: 24


Step,Training Loss,Validation Loss,Per Precision,Per Recall,Per F1,Loc Precision,Loc Recall,Loc F1,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
12,No log,1.495478,0.023102,0.155556,0.040230,0.001445,0.090909,0.002845,0.008040,0.142857,0.015224,0.260956
24,No log,1.297239,0.022936,0.111111,0.038023,0.001486,0.090909,0.002924,0.006734,0.107143,0.012672,0.486388
36,No log,0.984283,0.000000,0.000000,0.000000,0.002740,0.090909,0.005319,0.002439,0.017857,0.004292,0.819389
48,No log,0.601039,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.974768


In [ ]:
model = AutoModelForTokenClassification.from_pretrained("/content/drive/MyDrive/NLP_Project_New/models/bert_base_german_cased_finetuned_3ep")

In [ ]:
token_classifier = pipeline(
    "token-classification", model=model, tokenizer=tokenizer, aggregation_strategy="first"
)

In [ ]:
test_texts, test_labels = functions.read_conll_data('/content/drive/MyDrive/NLP_Project_New/data/train_test_val/test.tsv')

In [ ]:
all_sentences = []

for text in test_texts:
  sentence = " ".join(text)
  all_sentences.append(sentence)

In [ ]:
all_predictions = []
for sentence in all_sentences:
  prediction = token_classifier(sentence)
  all_predictions.append(prediction)

KeyboardInterrupt: 

In [ ]:
def list_transformer_results(sentence: str, predictions: list) -> tuple:
  words = sentence.split(" ")

  results = []

  running_char = 0
  for word in words:
    found_match = next((d for d in found_entities if d.get("start") == running_char), None)
    if found_match:

      results.append(found_match.get("entity_group"))
    else:
      results.append("O")
    running_char += len(word) + 1

  return words, results

In [ ]:
tokens = []
labels = []
predictions = []
for text, prediction, labels_list in zip(all_sentences, all_predictions, test_labels):
  word_list, prediction_list = list_results(text, prediction)
  tokens.extend(word_list)
  labels.extend(labels_list)
  predictions.extend(prediction_list)

In [ ]:
prediction_comparison = pd.DataFrame(
    {"TOKEN": tokens,
     "NER": labels,
     "bert_base_german_cased_finetuned_3ep": predictions
     })

In [ ]:
prediction_comparison.head()

,TOKEN,NER,bert_base_german_cased_finetuned_3ep
0,Darum,O,O
1,auch,O,O
2,der,O,O
3,selige,O,O
4,Lutherus,PER,PER


In [ ]:
simple_ne = {"B-PER": "PER", "I-PER": "PER", "B-LOC": "LOC", "I-LOC": "LOC", "O": "O"}

In [ ]:
prediction_comparison["NER"] = prediction_comparison["NER"].apply(lambda x: simple_ne[x])

In [ ]:
prediction_comparison

,TOKEN,NER,bert_base_german_cased_finetuned_3ep
0,Darum,O,O
1,auch,O,O
2,der,O,O
3,selige,O,O
4,Lutherus,PER,PER
...,...,...,...
157045,Verbot,O,O
157046,in,O,O
157047,sich,O,O
157048,halten,O,O
